In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv("../DATA/4-hours Pepperstone/4-hours_EURUSD.csv")

In [10]:
def displacement_detection(df, type_range="standard", strengh=3, period=100):
    """
    This function calculates and adds a 'displacement' column to a provided DataFrame. Displacement is determined based on
    the 'candle_range' which is calculated differently according to the 'type_range' parameter. Then, it calculates the
    standard deviation of the 'candle_range' over a given period and sets a 'threshold'. If 'candle_range' exceeds this 'threshold',
    a displacement is detected and marked as 1 in the 'displacement' column.

    Parameters:
    df (pd.DataFrame): The DataFrame to add the columns to. This DataFrame should have 'open', 'close', 'high', and 'low' columns.
    type_range (str, optional): Defines how to calculate 'candle_range'. 'standard' calculates it as the absolute difference between
                                'close' and 'open', 'extremum' calculates it as the absolute difference between 'high' and 'low'.
                                Default is 'standard'.
    strengh (int, optional): The multiplier for the standard deviation to set the 'threshold'. Default is 3.
    period (int, optional): The period to use for calculating the standard deviation. Default is 100.

    Returns:
    pd.DataFrame: The original DataFrame, but with four new columns: 'candle_range', 'MSTD', 'threshold' and 'displacement'.

    Raises:
    ValueError: If an unsupported 'type_range' is provided.
    """
    df_copy = df.copy()

    # Choose your type_range
    if type_range == "standard":
        df_copy["candle_range"] = np.abs(df_copy["close"] - df_copy["open"])
    elif type_range == "extremum":
        df_copy["candle_range"] = np.abs(df_copy["high"] - df_copy["low"])
    else:
        raise ValueError("Put a right format of type range")

    # Compute the STD of the candle range
    df_copy["MSTD"] = df_copy["candle_range"].rolling(period).std()
    df_copy["threshold"] = df_copy["MSTD"] * strengh

    # Displacement if the candle range is above the threshold
    df_copy["displacement"] = np.nan
    df_copy.loc[df_copy["threshold"] < df_copy["candle_range"], "displacement"] = 1
    df_copy["variation"] = df_copy["close"] - df_copy["open"]

    # Specify the way of the displacement
    df_copy["green_displacement"] = 0
    df_copy["red_displacement"] = 0

    df_copy.loc[(df_copy["displacement"] == 1) & (0 < df_copy["variation"]), "green_displacement"] = 1
    df_copy.loc[(df_copy["displacement"] == 1) & (df_copy["variation"] < 0), "red_displacement"] = 1

    # Shift by one because we only know that we have a displacement at the end of the candle (BE CAREFUL)
    df_copy["green_displacement"] = df_copy["green_displacement"].shift(1)
    df_copy["red_displacement"] = df_copy["red_displacement"].shift(1)

    df_copy["high_displacement"] = np.nan
    df_copy["low_displacement"] = np.nan

    df_copy.loc[df_copy["displacement"] == 1, "high_displacement"] = df_copy["high"]
    df_copy.loc[df_copy["displacement"] == 1, "low_displacement"] = df_copy["low"]

    df_copy["high_displacement"] = df_copy["high_displacement"].fillna(method="ffill")
    df_copy["low_displacement"] = df_copy["low_displacement"].fillna(method="ffill")

    return df_copy

In [11]:
df = displacement_detection(df)

In [12]:
df

,time,open,high,low,close,tick_volume,low_time,high_time,candle_range,MSTD,threshold,displacement,variation,green_displacement,red_displacement,high_displacement,low_displacement
0,2015-01-02 12:00:00,1.20467,1.20600,1.20347,1.20377,14827,2015-01-02 15:36:00,2015-01-02 12:48:00,0.00090,NaN,NaN,NaN,-0.00090,NaN,NaN,NaN,NaN
1,2015-01-02 16:00:00,1.20377,1.20437,1.20069,1.20131,19160,2015-01-02 19:36:00,2015-01-02 16:00:00,0.00246,NaN,NaN,NaN,-0.00246,0.0,0.0,NaN,NaN
2,2015-01-02 20:00:00,1.20133,1.20154,1.20011,1.20029,9335,2015-01-02 23:12:00,2015-01-02 20:00:00,0.00104,NaN,NaN,NaN,-0.00104,0.0,0.0,NaN,NaN
3,2015-01-03 00:00:00,1.20030,1.20055,1.19997,1.20014,1351,2015-01-03 00:24:00,2015-01-03 00:00:00,0.00016,NaN,NaN,NaN,-0.00016,0.0,0.0,NaN,NaN
4,2015-01-05 00:00:00,1.19454,1.19755,1.18642,1.19624,13941,2015-01-05 00:48:00,2015-01-05 03:36:00,0.00170,NaN,NaN,NaN,0.00170,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13671,2023-09-19 16:00:00,1.06997,1.07014,1.06785,1.06848,13607,2023-09-19 19:36:00,2023-09-19 16:00:00,0.00149,0.001416,0.004248,NaN,-0.00149,0.0,0.0,1.0747,1.06555
13672,2023-09-19 20:00:00,1.06848,1.06918,1.06761,1.06790,6544,2023-09-19 22:00:00,2023-09-19 20:24:00,0.00058,0.001416,0.004247,NaN,-0.00058,0.0,0.0,1.0747,1.06555
13673,2023-09-20 00:00:00,1.06790,1.06860,1.06768,1.06849,2478,2023-09-20 00:00:00,2023-09-20 03:36:00,0.00059,0.001416,0.004249,NaN,0.00059,0.0,0.0,1.0747,1.06555
13674,2023-09-20 04:00:00,1.06849,1.06897,1.06783,1.06786,4333,2023-09-20 06:24:00,2023-09-20 05:12:00,0.00063,0.001417,0.004252,NaN,-0.00063,0.0,0.0,1.0747,1.06555
